In [2]:
import sys
sys.path.append('..')
import torch
import torch.nn as nn
from torch.autograd import Variable
import models.resnet
import models.squeezenet
import torchaudio.transforms as tat
import torchvision.transforms as tvt
import spl_transforms
from loader_voxforge import *


In [8]:
DATA_PATH = "../data/voxforge"
LANGUAGES = ["de", "en", "es"]
FREQ_BANDS = 224
BATCH_SIZE = 10
MODEL_NAME = "resnet34"
MODEL_PATH = "../output/states/resnet34_aws_gpu_b100/all_layers/model_resnet34_55.pt"

In [3]:
# use_cuda
use_cuda = torch.cuda.is_available()
print("CUDA: {}".format(use_cuda))

CUDA: False


In [7]:
# Data
vx = VOXFORGE(DATA_PATH, langs=LANGUAGES, label_type="lang", use_cache=False)
#vx.find_max_len()
vx.maxlen = 150000
T = tat.Compose([
        tat.PadTrim(vx.maxlen),
        tat.MEL(n_mels=FREQ_BANDS),
        tat.BLC2CBL(),
        tvt.ToPILImage(),
        tvt.Scale((FREQ_BANDS, FREQ_BANDS)),
        tvt.ToTensor(),
    ])
TT = spl_transforms.LENC(vx.LABELS)
vx.transform = T
vx.target_transform = TT
dl = data.DataLoader(vx, batch_size = BATCH_SIZE, shuffle=True)


In [9]:
model_name = MODEL_NAME
model_path = MODEL_PATH
use_pretrained = model_path is None

MODELS = {
    "resnet34": {
        "model": models.resnet.resnet34(use_pretrained, num_langs=5),
        "state_dict": model_path,
    },
    "squeezenet": {
        "model": models.squeezenet.squeezenet(use_pretrained, num_langs=5),
        "state_dict": model_path,
    }
}

# Model and Loss
if model_name not in MODELS:
    model_name = "resnet34"
model = MODELS[model_name]["model"]
if not use_pretrained:
    model.load_state_dict(torch.load(MODELS[model_name]["state_dict"], map_location=lambda storage, loc: storage))
print(model)
criterion = nn.CrossEntropyLoss()

if use_cuda:
    model = model.cuda()

Sequential (
  (0): Conv2d(1, 3, kernel_size=(1, 1), stride=(1, 1))
  (1): ResNet (
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu): ReLU (inplace)
    (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (layer1): Sequential (
      (0): BasicBlock (
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (relu): ReLU (inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      )
      (1): BasicBlock (
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (relu): ReLU (inplace)
        (c

In [10]:
def predict():
    model.eval()
    vx.set_split("test")
    running_test_loss = 0
    correct = 0
    for i, (mb_test, tgts_test) in enumerate(dl):
        if use_cuda:
            mb_test, tgts_test = mb_test.cuda(), tgts_test.cuda()
        mb_test, tgts_test = Variable(mb_test), Variable(tgts_test)
        out_test = model(mb_test)
        predictions = out_test.data.max(1)[1]
        #if i % 10 == 0:
        #    print(predictions, tgts_test.data)
        loss_test = criterion(out_test, tgts_test)
        running_test_loss += loss_test.data[0]
        correct += (predictions == tgts_test.data).sum()
    print("loss: {}, acc: {}".format(running_test_loss, correct / len(vx)))

predict()


loss: 203.36296451091766, acc: 0.24549549549549549
